# Process manager

The process manager allows the construction of sets of tasks to be run, which can be (but are not limited to) launch files or ROS nodes to bring up a system that may include an instance of Gazebo.
Each instance of the process manager can be set with a different `ROS_MASTER_URI` and `GAZEBO_MASTER_URI`, so that multiple `roscore` and `gazebo` instances can be initialized on the same host at the same time.

## Starting a process manager with default parameters

In [1]:
from pcg_gazebo.task_manager import ProcessManager
# A process manager can be started by itself with an empty list of tasks
process_manager = ProcessManager()
# At this point neither Gazebo or roscore are running
# When no arguments are given, the default hostname and port
# arguments are going to be used.
print('ROS network configuration:')
print(process_manager.ros_config)

ROS network configuration:
ROS_MASTER_URI=http://localhost:11311, GAZEBO_MASTER_URI=http://localhost:11345


In [2]:
# At first, no tasks are available in the tasks list
print(process_manager.get_task_list())

[]


In [3]:
print('Check all the process stages available')
print(process_manager.stages)

Check all the process stages available
OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564f0d59e8>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564f0d5a20>)])


In [4]:
# You can start roscore by calling the method below
process_manager.create_ros_core_task()

True

In [5]:
# Now the roscore task can be found in the list
print(process_manager.get_task_list())

['roscore']


In [6]:
# The task can be started using the method below
# Running individual tasks will not use the stage order 
process_manager.run_task('roscore')

In [7]:
# Check if the task is running
print('Is task running? {}'.format(process_manager.is_task_running('roscore')))

Is task running? True


In [8]:
# Creating an RViz task with a timeout
# The required flag, like the flag for ROS nodes, says that once this task dies, 
# all other tasks must be killed.
# The process timeout starts a timer and will kill the task in the amount of seconds
# given by process_timeout
# IMPORTANT: process_timeout is based on the machine clock, not the simulation clock
# process_timeout=None means that the process will run without a timeout
process_manager.create_rviz_task(required=True, process_timeout=10)
print(process_manager.get_task_list())

['roscore', 'rviz']


In [9]:
# After 10 seconds all tasks will be killed along with rviz
process_manager.run_task('rviz')
process_manager.wait()


In [10]:
process_manager.kill_all_tasks()
del process_manager

## Running Gazebo

Similar to RViz, Gazebo can also be started. The same process will be repeated to start Gazebo with the empty world scenario.

In [11]:
# At first, no tasks are available in the tasks list
# A process manager can be started by itself with an empty list of tasks
# When ros_port and/or gazebo_port is given as None, a random port will be chosen
process_manager = ProcessManager(ros_port=None, gazebo_port=None)
print('ROS network configuration:')
print(process_manager.ros_config)

ROS network configuration:
ROS_MASTER_URI=http://localhost:18621, GAZEBO_MASTER_URI=http://localhost:25848


In [12]:
# A Gazebo task can also be started with a process timeout
process_manager.create_gazebo_empty_world_task(required=True, process_timeout=10)
print('Check all tasks available')
print(process_manager.get_task_list())
print('Check all the process stages available')
print(process_manager.stages)

Check all tasks available
['gazebo']
Check all the process stages available
OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f56902dd5c0>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564efd12b0>), ('gazebo', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564efd1b70>)])


In [13]:
process_manager.run_all_tasks()
# A roscore stage should be automatically added to the process manager list
print(process_manager.stages)
process_manager.wait()


OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f56902dd5c0>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564efd12b0>), ('gazebo', <pcg_gazebo.task_manager.stage.Stage object at 0x7f564efd1b70>)])


In [14]:
process_manager.kill_all_tasks()
del process_manager


In [15]:
# But a task that contains a Gazebo instance can also be started with a simulation timeout
# meaning that the process will be killed only when the simulation time reaches a timeout
# At first, no tasks are available in the tasks list
# A process manager can be started by itself with an empty list of tasks
# When ros_port and/or gazebo_port is given as None, a random port will be chosen
process_manager = ProcessManager(ros_port=None, gazebo_port=None)
print('ROS network configuration:')
print(process_manager.ros_config)

process_manager.create_gazebo_empty_world_task(required=True, simulation_timeout=10)
process_manager.run_all_tasks()
process_manager.wait()

ROS network configuration:
ROS_MASTER_URI=http://localhost:17899, GAZEBO_MASTER_URI=http://localhost:26148


In [16]:
print('Is task running? {}'.format(process_manager.is_task_running('gazebo')))

Is task running? False


In [17]:
# End the simulation by killing the Gazebo task
process_manager.kill_all_tasks()